<div style="padding: 2.25rem 2.5rem; border-radius: 18px; background: linear-gradient(120deg, #0b1220, #132c55); color: #f4f7fb; box-shadow: 0 18px 40px rgba(4,19,43,0.45);">
  <h1 style="margin-top: 0; font-size: 2.4rem; letter-spacing: 0.02em;">Live Trading Modular Reference</h1>
  <p style="font-size: 1.05rem; line-height: 1.6; max-width: 960px;">
    This notebook is a field guide for the live Alpaca trading bot. Use it during <strong>real market sessions</strong> to inspect configuration, verify broker connectivity, and launch the live data stream without touching simulation tooling.
  </p>
  <div style="display: flex; flex-wrap: wrap; gap: 1rem;">
    <div style="flex: 1 1 240px; background: rgba(255,255,255,0.06); border-radius: 14px; padding: 1rem 1.2rem;">
      <strong>Modular Focus</strong>
      <p style="margin: .35rem 0 0; font-size: .95rem;">Strategy config · Broker facade · Indicators · Risk</p>
    </div>
    <div style="flex: 1 1 240px; background: rgba(255,255,255,0.06); border-radius: 14px; padding: 1rem 1.2rem;">
      <strong>Audience</strong>
      <p style="margin: .35rem 0 0; font-size: .95rem;">Operators running the bot against live Alpaca endpoints</p>
    </div>
    <div style="flex: 1 1 240px; background: rgba(255,255,255,0.06); border-radius: 14px; padding: 1rem 1.2rem;">
      <strong>Safety First</strong>
      <p style="margin: .35rem 0 0; font-size: .95rem;">Review every step before transmitting an order</p>
    </div>
  </div>
</div>

---

### 🔗 Quick Navigation
- [1. Environment prerequisites](#1-environment-prerequisites)
- [2. Strategy configuration anatomy](#2-strategy-configuration-anatomy)
- [3. Credential handling & broker facade](#3-credential-handling-and-broker-facade)
- [4. Indicators snapshot](#4-indicators-module-snapshot)
- [5. Risk manager overview](#5-risk-manager-overview)
- [6. Strategy orchestration hooks](#6-strategy-orchestration-hooks)
- [7. Live stream wiring](#7-live-stream-wiring-manual-start)


<div style="padding: 1.25rem 1.35rem; border-left: 6px solid #ef4444; background: rgba(239, 68, 68, 0.08); border-radius: 0 14px 14px 0;">⚠️ <strong>Live Trading Notice:</strong> Cells in this notebook instantiate <em>real</em> broker clients. Confirm credentials, buying power, and open positions before executing anything that places orders. </div>

## 1. Environment prerequisites
<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="padding: 1.2rem 1.4rem; border-left: 6px solid #4c9aff; background: rgba(76,154,255,0.08); border-radius: 0 14px 14px 0;">
    <strong>Goal:</strong> Authenticate the bot against Alpaca's live trading and market data endpoints.
  </div>
</div>

<table>
  <thead>
    <tr><th align="left">Step</th><th align="left">Action</th><th align="left">Notes</th></tr>
  </thead>
  <tbody>
    <tr><td>1</td><td>Export your API keys</td><td>Prefer <code>direnv</code> or secrets manager so they refresh automatically.</td></tr>
    <tr><td>2</td><td>Verify account mode</td><td><code>APCA_API_BASE_URL</code> must point to the live trading URL.</td></tr>
    <tr><td>3</td><td>Double-check equity</td><td>Use the REST client (<code>get_account()</code>) to confirm buying power.</td></tr>
  </tbody>
</table>

Use the cell below to materialize environment variables in the current kernel session. The helper prints the destination host to confirm you are not on the paper endpoint.

In [ ]:
from __future__ import annotations

import asyncio
import inspect
from dataclasses import asdict, fields

import nest_asyncio

from bootstrap import ensure_requirements
from broker import AlpacaBroker
from config import AppConfig, AlpacaCredentials, StrategyConfig
from indicators import IndicatorSet
from risk import RiskManager
from strategy import EmaSmaStrategy

# Guarantee compatibility with already-running event loops (common in notebooks)
nest_asyncio.apply()

# Safety: no automatic installation when running during market hours.
ensure_requirements(auto_install=False)

## 2. Strategy configuration anatomy

<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="flex: 1 1 260px; padding: 1rem 1.1rem; background: #111827; color: #f9fafb; border-radius: 14px;">
    <h4 style="margin-top: 0;">Why it matters</h4>
    <p style="margin-bottom: 0; font-size: .95rem; line-height: 1.55;">The <code>StrategyConfig</code> dataclass orchestrates symbols, bar windows, and risk parameters. Keeping it transparent lets you audit live runs rapidly.</p>
  </div>
  <div style="flex: 1 1 260px; padding: 1rem 1.1rem; background: #132c55; border-radius: 14px; border: 1px solid #132c55; color: #f9fafb;">
    <h4 style="margin-top: 0;">Refresh cadence</h4>
    <p style="margin-bottom: 0; font-size: .95rem; line-height: 1.55;">Re-evaluate settings before the open, after major macro events, or when swapping strategies.</p>
  </div>
</div>

| Attribute | Purpose | Live-trading tip |
| --- | --- | --- |
| `symbols` | Tuple of tradable tickers | Keep correlated hedges adjacent for faster updates. |
| `cash_alloc_pct` | Fraction of equity allocated | Cap below 0.9 to reserve dry powder for slippage. |
| `ema_fast` / `ema_slow` | Trend detection windows | Shorten fast EMA in high-volatility regimes. |
| `sma_window` | Mean reversion baseline | Larger windows smooth noise but slow reactions. |
| `atr_window` | Volatility sizing | Inspect ATR values when adjusting stop distances. |

Use the next cell to surface the current configuration and confirm its runtime values.

In [4]:
strategy_config = StrategyConfig()
for field in fields(strategy_config):
    value = getattr(strategy_config, field.name)
    print(f"{field.name}: {value}")

NameError: name 'StrategyConfig' is not defined

<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="padding: 1.2rem 1.4rem; border-left: 6px solid #4c9aff; background: rgba(76,154,255,0.08); border-radius: 0 14px 14px 0;">
    <strong>Live tweak shortcut:</strong> Override parameters here (e.g., swap tickers or throttle allocation) <em>before</em> instantiating dependent modules below.
  </div>
</div>

In [ ]:
custom_strategy = StrategyConfig(
    bullish_symbol="SOXL",
    bearish_symbol="SOXS",
    allocation=strategy_config.allocation,
    indicators=strategy_config.indicators,
    risk=strategy_config.risk,
    stream=strategy_config.stream,
)
custom_strategy.allocation.cash_fraction = 0.75  # allocate 75% of buying power
custom_strategy.stream.symbols = ["SOXL", "SOXS"]
custom_strategy

## 3. Credential handling and broker facade

<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="flex: 1 1 260px; padding: 1rem 1.1rem; background: #111827; color: #f9fafb; border-radius: 14px;">
    <h4 style="margin-top: 0;">Broker toolkit</h4>
    <ul style="padding-left: 1.1rem; margin: .4rem 0 0;">
      <li><code>submit_order()</code> &amp; <code>replace_order()</code></li>
      <li>Live polygon/Alpaca market data stream</li>
      <li>Order-state callbacks for fills and cancels</li>
    </ul>
  </div>
  <div style="flex: 1 1 260px; padding: 1rem 1.1rem; background: #132c55; border-radius: 14px; border: 1px solid #132c55; color: #f9fafb;">
    <h4 style="margin-top: 0;">Security checklist</h4>
    <p style="margin: 0; font-size: .95rem; line-height: 1.55;">Rotate API keys regularly and scope them to trading + data only. Never commit credentials—reference environment variables exclusively.</p>
  </div>
</div>

Inspect the broker instance below to confirm endpoints, subscribed symbols, and current throttle limits before placing live orders.


In [ ]:
credentials = AlpacaCredentials.from_env()
app_config = AppConfig(credentials=credentials, strategy=custom_strategy)
broker = AlpacaBroker(app_config.credentials, app_config.strategy)

# Summarize the key callable broker methods for quick reference.
broker_operations = {
    name: obj
    for name, obj in inspect.getmembers(broker, predicate=callable)
    if not name.startswith("_")
}
list(broker_operations.keys())

<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="padding: 1.2rem 1.4rem; border-left: 6px solid #4c9aff; background: rgba(76,154,255,0.08); border-radius: 0 14px 14px 0;">
    <strong>Pro tip:</strong> Use <code>print_signature(broker.submit_order)</code> to display argument defaults, ensuring position sizes and time-in-force match your playbook before the opening bell.
  </div>
</div>

In [ ]:
for name in [
    "get_account",
    "get_positions",
    "submit_market_order",
    "close_position",
    "close_all_positions",
    "subscribe_bars",
    "run_stream",
]:
    func = broker_operations[name]
    signature = str(inspect.signature(func))
    doc = inspect.getdoc(func) or "No docstring."
    print(f"{name}{signature}
  {doc}
")

## 4. Indicators module snapshot
<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="padding: 1.2rem 1.4rem; border-left: 6px solid #4c9aff; background: rgba(76,154,255,0.08); border-radius: 0 14px 14px 0;">
    <p style="margin: 0; font-size: .97rem; line-height: 1.6;">
      <strong>IndicatorSet</strong> keeps EMA, SMA, and ATR metrics synchronized with the latest bars. With live data flowing, review the rolling values here to validate the signal regime you're trading in.
    </p>
  </div>
</div>

| Metric | Description | Operational use |
| --- | --- | --- |
| EMA fast/slow | Trend accelerators | Confirm crossovers align with your manual charting. |
| SMA baseline | Long-term drift | Use as bias filter to avoid counter-trend orders. |
| ATR window | Volatility proxy | Scale stop-losses and position sizes responsively. |

In [ ]:
indicators = IndicatorSet.from_config(
    ema_period=app_config.strategy.indicators.ema_period,
    sma_period=app_config.strategy.indicators.sma_period,
    atr_period=app_config.strategy.indicators.atr_period,
)

seed_bars = broker.get_seed_bars(
    app_config.strategy.bullish_symbol,
    app_config.strategy.indicators.seed_bars,
)
indicators.seed_from_bars(seed_bars)

print("Latest EMA:", indicators.latest_ema)
print("Latest SMA:", indicators.latest_sma)

## 5. Risk manager overview

<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="flex: 1 1 260px; padding: 1rem 1.1rem; background: #111827; color: #f9fafb; border-radius: 14px;">
    <h4 style="margin-top: 0;">Automated guardrails</h4>
    <ul style="margin: .4rem 0 0; font-size: .95rem; line-height: 1.55;">
      <li>Flatten near close</li>
      <li>Buying power validation</li>
      <li>Dynamic hedge toggles</li>
    </ul>
  </div>
  <div style="flex: 1 1 260px; padding: 1rem 1.1rem; background: #132c55; border-radius: 14px; border: 1px solid #132c55; color: #f9fafb;">
    <h4 style="margin-top: 0;">Operator checklist</h4>
    <ul style="margin: .4rem 0 0; font-size: .95rem; line-height: 1.55;">
      <li>Inspect max drawdown thresholds</li>
      <li>Confirm trading hours boundaries</li>
      <li>Simulate risk adjustments on small size first</li>
    </ul>
  </div>
</div>

Use the following cell to introspect callable methods and adjust risk toggles while monitoring fills.


In [ ]:
risk_manager = RiskManager(broker, app_config.strategy.risk)

risk_summary = asdict(app_config.strategy.risk)
print("Risk configuration:")
for key, value in risk_summary.items():
    print(f"  {key}: {value}")

# Display coroutine signatures for situational awareness.
for name, func in inspect.getmembers(risk_manager, predicate=callable):
    if name.startswith("_"):
        continue
    signature = str(inspect.signature(func))
    print(f"RiskManager.{name}{signature}")

## 6. Strategy orchestration hooks
<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="padding: 1.2rem 1.4rem; border-left: 6px solid #4c9aff; background: rgba(76,154,255,0.08); border-radius: 0 14px 14px 0;">
      <code>EmaSmaStrategy</code> contains the heart of the crossover logic. The helper below lets you explore lifecycle hooks—<code>on_bar</code>, <code>on_fill</code>, and <code>rebalance_positions</code>—so you can inject logging or additional trade management while markets run.
  </div>
</div>

In [ ]:
strategy = EmaSmaStrategy(broker, indicators, risk_manager)

print("Strategy is monitoring:", app_config.strategy.all_symbols())
print("Attributes:", [name for name in strategy.__dict__.keys() if not name.startswith("_")])
print("Public coroutine methods:")
for name, func in inspect.getmembers(strategy, predicate=callable):
    if name.startswith("_"):
        continue
    print(" -", name, inspect.signature(func))

## 7. Live stream wiring (manual start)
<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="padding: 1.25rem 1.35rem; border-left: 6px solid #ef4444; background: rgba(239, 68, 68, 0.08); border-radius: 0 14px 14px 0;">
    <strong>Final gate:</strong> Only execute the streaming cell once all prior inspections are complete and you're ready to react to live bars. Stop with <code>await broker.stop_stream()</code> to pause trading.
  </div>
</div>

<div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-top: 1rem;">
  <div style="flex: 1 1 260px; padding: 1rem 1.1rem; background: #111827; color: #f9fafb; border-radius: 14px;">
    <strong>Stream Checklist</strong>
    <ul style="margin: .5rem 0 0; padding-left: 1.1rem; line-height: 1.55;">
      <li>Broker authenticated</li>
      <li>Strategy hooks tested</li>
      <li>Risk boundaries verified</li>
      <li>Logging tailing in terminal</li>
    </ul>
  </div>
  <div style="flex: 1 1 260px; padding: 1rem 1.1rem; background: #132c55; border-radius: 14px; border: 1px solid #132c55; color: #f9fafb;">
    <strong>What to monitor</strong>
    <ul style="margin: .5rem 0 0; padding-left: 1.1rem; line-height: 1.55;">
      <li>Fill latency vs. exchange timestamps</li>
      <li>Indicator drift between bars</li>
      <li>Risk manager overrides triggering</li>
    </ul>
  </div>
</div>


In [ ]:
async def handle_bar(bar):
    await strategy.on_bar(bar)

broker.subscribe_bars(handle_bar, *app_config.strategy.all_symbols())

# Uncomment the line below to start streaming live market data.
# await broker.run_stream()